# 卷积神经网络开放题

# 数据集
本次开放题将与课程内容保持一致，将使用图像数据集Fashion-MNIST [1] 进行计算机视觉任务的设计，该数据集由衣服、鞋子等服饰组成，共10个类别。

这里简介将此数据集转换成卷积神经网络所需要的输入格式的方法：

## 加载数据集

首先导入本作业需要的包或模块。

In [8]:
import torchvision
import torch
from matplotlib import pyplot as plt
from IPython import display

通过`load_data_fashion_mnist`函数对数据集进行加载，另外函数还指定了参数`transform = transforms.ToTensor()`使所有数据转换为`Tensor`，如果不进行转换则返回的是PIL图片。`transforms.ToTensor()`将尺寸为 (H x W x C) 且数据位于 [0, 255] 的PIL图片或者数据类型为`np.uint8`的NumPy数组转换为尺寸为 (C x H x W) 且数据类型为`torch.float32`且位于 [0.0, 1.0] 的`Tensor`。

我们将在训练数据集上训练模型，并将训练好的模型在测试数据集上评价模型的表现。函数中`mnist_train`是`torch.utils.data.Dataset`的子类，所以我们可以将其传入`torch.utils.data.DataLoader`来创建一个读取小批量数据样本的`DataLoader`实例。

在实践中，数据读取经常是训练的性能瓶颈，特别当模型较简单或者计算硬件性能较高时。PyTorch的`DataLoader`中一个很方便的功能是允许使用多进程来加速数据读取。这里我们通过参数`num_workers`来设置进程读取数据。

In [2]:
def load_data_fashion_mnist(batch_size, resize=None, root="/home/kesci/input/FashionMNIST2065/"):
    """Download the fashion mnist dataset and then load into memory."""
    trans = []
    if resize:
        trans.append(torchvision.transforms.Resize(size=resize))
    trans.append(torchvision.transforms.ToTensor())

    transform = torchvision.transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, download=True, transform=transform)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, download=True, transform=transform)

    train_iter = torch.utils.data.DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
    test_iter = torch.utils.data.DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

    return train_iter, test_iter

对于AlexNet，我们需要将 Fashion-MNIST 数据集的图像高和宽扩大到224，这个可以通过在`load_data_fashion_mnist`中传入`Resize`来实现，这边设置数据的`batch_size`为128。

In [3]:
batch_size = 128
# 如出现“out of memory”的报错信息，可减小batch_size或resize
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=224)

Fashion-MNIST中一共包括了10个类别，分别为t-shirt（T恤）、trouser（裤子）、pullover（套衫）、dress（连衣裙）、coat（外套）、sandal（凉鞋）、shirt（衬衫）、sneaker（运动鞋）、bag（包）和ankle boot（短靴）。以下函数可以将数值标签转成相应的文本标签。

In [4]:
def get_fashion_mnist_labels(labels):
    text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
                   'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
    return [text_labels[int(i)] for i in labels]

下面定义一个可以在一行里画出多张图像和对应标签的函数。

In [5]:
def show_fashion_mnist(images, labels):
    """Use svg format to display plot in jupyter"""
    display.set_matplotlib_formats('svg')
    # 这里的_表示我们忽略（不使用）的变量
    _, figs = plt.subplots(1, len(images), figsize=(12, 12))
    for f, img, lbl in zip(figs, images, labels):
        f.imshow(img.view((224, 224)).numpy())
        f.set_title(lbl)
        f.axes.get_xaxis().set_visible(False)
        f.axes.get_yaxis().set_visible(False)

读取训练数据集中第一个`batch`的数据。

In [6]:
train_data = iter(train_iter)
images, labels = next(train_data)

现在，我们看一下训练数据集中前10个样本的图像内容和文本标签。

In [7]:
labels = get_fashion_mnist_labels(labels)
show_fashion_mnist(images[:10], labels[:10])
plt.show()

<Figure size 864x864 with 10 Axes>

# 卷积神经网络（CNN）

对于计算机视觉的分类任务，在很长一段时间里流行的是研究者通过经验与智慧所设计并生成的手工特征。这类图像分类研究的主要流程是：

1. 获取图像数据集；
2. 使用已有的特征提取函数生成图像的特征；
3. 使用机器学习模型对图像的特征分类。

卷积神经网络就是含卷积层的神经网络，深度卷积神经网络的兴起改变了计算机视觉任务中手工设计的特征的传统，引领了诸多影响深远的研究。

## LeNet



LeNet [2] 作为一个早期用来识别手写数字图像的卷积神经网络，展示了通过梯度下降训练卷积神经网络可以达到手写数字识别在当时最先进的结果。如下图所示：


![LeNet模型](https://d2l.ai/_images/lenet.svg)


LeNet的模型结构分为卷积层块和全连接层块两个部分：

- 卷积层保留输入形状，使图像的像素在高和宽两个方向上的相关性均可能被有效识别，并且通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。

- 全连接层块将卷积层块的输出中每个样本变平（flatten），即输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，从而进行分类。



## AlexNet

2012年，AlexNet [3] 横空出世，使用了8层卷积神经网络，并以很大的优势赢得了ImageNet 2012图像识别挑战赛。

AlexNet与LeNet的设计理念非常相似，但相对较小的LeNet相比，AlexNet包含5层卷积和2层全连接隐藏层，以及1个全连接输出层，模型参数也大大增加。由于早期显存的限制，最早的AlexNet使用双数据流的设计，使一个GPU只需要处理一半模型。如下图所示：

![AlexNet模型](https://tangshusen.me/Dive-into-DL-PyTorch/img/chapter05/5.6_alexnet.png)

AlexNet首次证明了神经网络以端到端（end-to-end）的方式学习到的特征可以超越手工设计的特征，从而一举打破计算机视觉研究的前状。



### 问题一：
- 阅读提出上述两种网络的相关论文，试从数据集的预处理、激活函数的使用、训练方法的改进以及模型结构的变化等角度，从理论层面分析比较LeNet与AlexNet的结构差异，并尝试解释AlexNet为什么会具有对计算机视觉任务优越的处理性能。
- AlexNet对Fashion-MNIST数据集来说可能过于复杂，请尝试对模型进行简化来使训练更快，同时保证分类准确率（accuracy）不明显下降（不低于85%），并将简化后的结构、节省的训练时间以及下降的准确率等相关指标以表格的形式进行总结分析。

## VGG

AlexNet在LeNet的基础上增加了3个卷积层，同时对网络的卷积窗口、输出通道数和构造顺序均做了大量的调整。AlexNet指明了深度卷积神经网络可以取得出色的结果，基于这一个理念，牛津大学的实验室Visual Geometry Group实验室提出了VGG [4] 网络，提供了通过重复使用简单的基础块来构建深度模型的思路。VGG每个基础块组成规律是：连续使用数个相同的填充为1、窗口形状为$3\times 3$的卷积层后接上一个步幅为2、窗口形状为$2\times 2$的最大池化层。卷积层保持输入的高和宽不变，而池化层则对其减半。如下图所示：

![VGG模型](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/vgg.jpg)

可以看到，每次经过基础块以后，网络会将输入的高和宽减半，直到最终高和宽变成7后传入全连接层。与此同时，输出通道数每次翻倍，直到变成512。因为每个卷积层的窗口大小一样，VGG这种高和宽减半以及通道翻倍的设计使得多数卷积层都有相同的模型参数尺寸和计算复杂度。

### 问题二：
- LeNet与AlexNet在接近图像输入的卷积模块中都引入了较大尺寸的卷积核，如$5\times 5$或者$7\times 7$的卷积窗口来捕捉更大范围的图像信息，试分析VGG每个基础块的固定设计是否会影响到图像的粗粒度信息提取，并且对比不同结构模块输出的特征图进行对比。
- 尝试将Fashion-MNIST中图像的高和宽由224改为96，试分析VGG网络的参数变化情况，并且对比模型训练时间、分类准确率（accuracy）等实验指标受到的影响，以表格的形式进行总结分析。

## NiN

之前介绍的LeNet、AlexNet和VGG在设计上的共同之处是：先以由卷积层构成的模块充分抽取空间特征，再以由全连接层构成的模块来输出分类结果。其中，AlexNet和VGG对LeNet的改进主要在于如何对这两个模块加宽（增加通道数）和加深。网络中的网络（NiN）[5] 提出了另外一个思路，即串联多个由卷积层和“全连接”层构成的小网络来构建一个深层网络。

卷积层的输入和输出通常是四维数组（样本，通道，高，宽），而全连接层的输入和输出则通常是二维数组（样本，特征）。如果想在全连接层后再接上卷积层，则需要将全连接层的输出变换为四维，$1\times 1$卷积层可以看成全连接层，其中空间维度（高和宽）上的每个元素相当于样本，通道相当于特征。因此，NiN使用$1\times 1$卷积层来替代全连接层，从而使空间信息能够自然传递到后面的层中去。下图对比了NiN同AlexNet和VGG等网络在结构上的主要区别。

![左图是AlexNet和VGG的网络结构局部，右图是NiN的网络结构局部](http://zh.d2l.ai/_images/nin.svg)

NiN块是NiN中的基础块。它由一个卷积层加两个充当全连接层的$1\times 1$卷积层串联而成。其中第一个卷积层的超参数可以自行设置，而第二和第三个卷积层的超参数一般是固定的。


## GoogLeNet

在2014年的ImageNet图像识别挑战赛中，一个名叫GoogLeNet的网络结构大放异彩 [6] ，它虽然在名字上向LeNet致敬，但在网络结构上已经很难看到LeNet的影子。GoogLeNet吸收了NiN中网络串联网络的思想，并在此基础上做了很大改进。GoogLeNet中的基础卷积块叫作Inception块，得名于同名电影《盗梦空间》（Inception）。与上NiN块相比，这个基础块在结构上更加复杂，如下图所示：

![Inception块的结构](http://zh.d2l.ai/_images/inception.svg)

Inception块里有4条并行的线路。前3条线路使用窗口大小分别是$1\times 1$、$3\times 3$和$5\times 5$的卷积层来抽取不同空间尺寸下的信息，其中中间2个线路会对输入先做$1\times 1$卷积来减少输入通道数，以降低模型复杂度。第四条线路则使用$3\times 3$最大池化层，后接$1\times 1$卷积层来改变通道数。4条线路都使用了合适的填充来使输入与输出的高和宽一致。最后将每条线路的输出在通道维上连结，并输入接下来的层中去。Inception块中可以自定义的超参数是每个层的输出通道数，以此来控制模型复杂度。

### 问题三：
- 对比AlexNet、VGG和NiN、GoogLeNet的模型参数尺寸，从理论的层面分析为什么后两个网络可以显著减小模型参数尺寸？
- GoogLeNet有数个后续版本，包括加入批量归一化层 [7]、对Inception块做调整 [8] 和加入残差连接 [9]，请尝试实现并运行它们，然后观察实验结果，以表格的形式进行总结分析。

## 参考文献
[1] Xiao, H., Rasul, K., & Vollgraf, R. (2017). Fashion-mnist: a novel image dataset for benchmarking machine learning algorithms. arXiv preprint arXiv:1708.07747.

[2] LeCun, Y., Bottou, L., Bengio, Y., & Haffner, P. (1998). Gradient-based learning applied to document recognition. Proceedings of the IEEE, 86(11), 2278-2324.

[3] Krizhevsky, A., Sutskever, I., & Hinton, G. E. (2012). Imagenet classification with deep convolutional neural networks. In Advances in neural information processing systems (pp. 1097-1105).

[4] Simonyan, K., & Zisserman, A. (2014). Very deep convolutional networks for large-scale image recognition. arXiv preprint arXiv:1409.1556.

[5] Lin, M., Chen, Q., & Yan, S. (2013). Network in network. arXiv preprint arXiv:1312.4400.

[6] Szegedy, C., Liu, W., Jia, Y., Sermanet, P., Reed, S., & Anguelov, D. & Rabinovich, A.(2015). Going deeper with convolutions. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 1-9).

[7] Ioffe, S., & Szegedy, C. (2015). Batch normalization: Accelerating deep network training by reducing internal covariate shift. arXiv preprint arXiv:1502.03167.

[8] Szegedy, C., Vanhoucke, V., Ioffe, S., Shlens, J., & Wojna, Z. (2016). Rethinking the inception architecture for computer vision. In Proceedings of the IEEE Conference on Computer Vision and Pattern Recognition (pp. 2818-2826).

[9] Szegedy, C., Ioffe, S., Vanhoucke, V., & Alemi, A. A. (2017, February). Inception-v4, inception-resnet and the impact of residual connections on learning. In Proceedings of the AAAI Conference on Artificial Intelligence (Vol. 4, p. 12).

## 项目报告
本次大作业的终审评估以项目报告作为重要依据，开放题报告的内容和排版要求请下载文件：


[termproject2.zip](https://boyuai.oss-cn-shanghai.aliyuncs.com/disk/YouthAI%E7%A7%8B%E5%AD%A3%E6%80%9D%E7%BB%B4%E7%8F%AD-%E4%B8%8A%E8%AF%BE%E8%A7%86%E9%A2%91/termproject2.zip)

需要注意的是，文件中：
- `termproject.pdf`提供了项目报告的内容格式要求
- `termproject_exp.pdf`提供了项目报告的内容排版样例


推荐使用`LaTeX`软件进行报告的撰写，相关`.tex`以及`.sty`源文件一并附于文件夹中。
